**Téléchargement de l'algorithme et installation des pré-requis**

In [29]:
%cd
!git clone --quiet https://github.com/matterport/Mask_RCNN.git
%cd ~/Mask_RCNN
!pip install -for h in range(len(corpus)):
!pip install -r requirements.txt
!pip install pillow
!python setup.py install

/root
fatal: destination path 'Mask_RCNN' already exists and is not an empty directory.
/root/Mask_RCNN
/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `pip install -for h in range(len(corpus)):'
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=1853b7232c6d3053dbb5d373a5e7bc754a02b70f5d32aa704741dce579ede956
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/__init__.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/config.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/parallel_model.py -> build/bdist.linux-x86_64/egg/mrcnn
byte-compiling build/bdist.linux-x86_64/egg/mrcnn/__init__.py to __init_

**Spécification des versions de TensorFlow et Keras**

In [ ]:
%tensorflow_version 1.x
!pip install q keras==2.1.5
# SI LA VERSION DE KERAS INSTALLEE EST  
# DIFFERENTE DE 2.1.5, RELANCER LA CELLULE

  Using cached https://files.pythonhosted.org/packages/ba/65/e4aff762b8696ec0626a6654b1e73b396fcc8b7cc6b98d78a1bc53b85b48/Keras-2.1.5-py2.py3-none-any.whl
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


**Modification de l'algorithme pour la reconnaissance d'illustrations**

In [ ]:
%cd ~/Mask_RCNN

!cp ~/Mask_RCNN/samples/balloon/balloon.py ./illustration.py

!sed -i -- 's/balloon/illustration/g' illustration.py
!sed -i -- 's/Balloon/Illustration/g' illustration.py
!sed -i -- 's/epochs=30/epochs=5/g' illustration.py

/root/Mask_RCNN


**Téléchargement du corpus- Spécifier un fichier ZIP nommé "corpus.zip" contenant les images sans dossier**

In [ ]:

import os
from zipfile import ZipFile
from shutil import copy
os.chdir('/root/Mask_RCNN')


os.makedirs('dataset')
os.chdir('dataset')

from google.colab import files
uploaded = files.upload()
fileName = "extraction.zip"
ds = ZipFile(fileName)
ds.extractall()
os.remove(fileName)
print('Extracted zip file ' + fileName)


Saving extraction.zip to extraction.zip
Extracted zip file extraction.zip


**Téléchargement des poids pré-entrainés sur le corpus Droits et Libertés depuis un Google Drive**

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.chdir('/root/Mask_RCNN')

fileId = "1H0swxkfgqTdUz5aNyIy_g3ixPUmT_jQW"

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': fileId})
downloaded.GetContentFile("droits_libertes_poids.h5")

**Préparation et configuration de l'algorithme**

In [ ]:
import os
import sys
import random
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("/")

import warnings
warnings.filterwarnings("ignore")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
import illustration

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join('', "droits_libertes_poids.h5")

# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):

    utils.download_trained_weights(COCO_MODEL_PATH)

# Directory of images to run detection on
IMAGE_DIR = os.path.join(ROOT_DIR, "dataset")

config = illustration.IllustrationConfig()

class InferenceConfig(config.__class__):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

Using TensorFlow backend.


**Création du modèle**

In [ ]:
model = modellib.MaskRCNN(mode="inference", model_dir='mask_rcnn_coco.hy', config=config)
# Load weights trained on MS-COCO
model.load_weights('droits_libertes_poids.h5', by_name=True)







Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.








**Création du corpus**

In [ ]:
# CORPUS avec des filenames
class_names = ['BG', 'illustration2']
import glob
corpus = [file for file in glob.glob("/root/Mask_RCNN/dataset/*.*")]
print(corpus)

['/root/Mask_RCNN/dataset/1950_114_006.JPG', '/root/Mask_RCNN/dataset/1950_114_005.JPG', '/root/Mask_RCNN/dataset/1950_114_004.JPG']


**Segmentation du corpus**

In [ ]:
import cv2
listederesultats = []
for image in glob.glob("/root/Mask_RCNN/dataset/*.*") :
  image = cv2.imread(image, cv2.IMREAD_COLOR)
  results = model.detect([image], verbose=0)
  listederesultats.append(results)

**Visualisation de la segmentation**

In [ ]:
for h in range(len(corpus)):
  page = cv2.imread(corpus[h], cv2.IMREAD_COLOR)
  r = listederesultats[h][0]
  visualize.display_instances(page, r['rois'], r['masks'], r['class_ids'], class_names, r['scores'])

Output hidden; open in https://colab.research.google.com to view.

**Extraction des illustrations en fichiers PNG**

In [ ]:
compteurimage = 1
compteur = 1

for h in range(len(corpus)):
  imaage = corpus[h]
  r = listederesultats[h][0]
  print("Image : ", compteurimage)
  masque = r["masks"]
  masque = masque.astype(int)
  for i in range(masque.shape[2]):
    print("illustration : ", i)
    temp = cv2.imread(imaage, cv2.IMREAD_COLOR)
    for j in range(temp.shape[2]):
      temp[:,:,j] = temp[:,:,j] * masque[:,:,i]
    tempgray = cv2.cvtColor(temp,cv2.COLOR_BGR2GRAY)
    _,alpha = cv2.threshold(tempgray,0,255,cv2.THRESH_BINARY)
    b, g, r = cv2.split(temp)
    rgba = [b,g,r, alpha]
    crop = cv2.merge(rgba,4)
    y,x = crop[:,:,3].nonzero() # get the nonzero alpha coordinates
    minx = np.min(x)
    miny = np.min(y)
    maxx = np.max(x)
    maxy = np.max(y) 
    cropImg = crop[miny:maxy, minx:maxx]
    nomimage = "extraction"+str(compteur)+"_crop.png"
    cv2.imwrite(nomimage,cropImg)
    compteur = compteur + 1
  compteurimage = compteurimage + 1


Image :  1
illustration :  0
illustration :  1
illustration :  2
illustration :  3
illustration :  4
illustration :  5
illustration :  6
illustration :  7
illustration :  8
Image :  2
illustration :  0
illustration :  1
Image :  3
illustration :  0
illustration :  1
